In [ ]:
import numpy as np
import random
import csv

# 超参数
alpha = 0.9                # 学习率
gamma = 0.9                # 折扣因子
delta = 1e-2               # 状态变化量
G_ref = 15                 # 目标函数 G 的参考值
G_min_initial = 20         # 最小目标函数值初始值
N_T = int(5e4)             # 最大训练回合数
N_i = 5000                 # 每个优化过程的训练次数
M = int(1e4)               # 基于 ε-greedy 法的探索次数

# DAB DC-DC 转换器电路参数
n = 1                      # 变压器匝数比
fs = 50e3                  # 开关频率，单位：Hz
Lk = 41e-6                 # 串联电感，单位：H
k = None                   # k 由输入 V1 和输出 V2 计算
Q_table = {}

# 动作空间：增量变化
actions = [(d1, d2, d3) for d1 in [0, delta, -delta] for d2 in [0, delta, -delta] for d3 in [0, delta, -delta]]

# 计算归一化系数
def update_normalization_factor(V2):
    return n * V2 / (4 * fs * Lk)

# 离散化步长和状态空间大小
num_bins = int(2 / delta) + 1  # 4001 个离散化值

# 离散化函数
def discretize(value):
    index = int(round((value + 1) / delta))
    return max(0, min(index, num_bins - 1))

# 根据离散化后的 D1, D2, D3 生成单一索引
def get_state_index(D1, D2, D3):
    D1_index = discretize(D1)
    D2_index = discretize(D2)
    D3_index = discretize(D3)
    return D1_index * num_bins * num_bins + D2_index * num_bins + D3_index

# Q 值更新函数
def update_q_value(state_index, action, reward, next_state_index):
    if state_index not in Q_table:
        Q_table[state_index] = np.zeros(len(actions))
    if next_state_index not in Q_table:
        Q_table[next_state_index] = np.zeros(len(actions))
    
    best_next_action = np.argmax(Q_table[next_state_index])
    td_target = reward + gamma * Q_table[next_state_index][best_next_action]
    td_error = td_target - Q_table[state_index][action]
    Q_table[state_index][action] += alpha * td_error

def determine_mode(D1, D2, D3):
    # 根据提供的不等式条件选择模式
    if D1 >= D2 and 0 <= D3 <= (D1 - D2):
        return "mode_1"
    elif D1 >= D2 and 0 <= D3 <= (D1 - D2):
        return "mode_1_prime"
    elif D2 >= D1 and (1 + D1 - D2) <= D3 <= 1:
        return "mode_2"
    elif D2 >= D1 and (1 + D1 - D2) <= D3 <= 1:
        return "mode_2_prime"
    elif D2 <= (1 - D1) and D1 <= D3 <= (1 - D2):
        return "mode_3"
    elif D2 <= (1 - D1) and D1 <= D3 <= (1 - D2):
        return "mode_3_prime"
    elif D1 <= D3 <= 1 and (1 - D3) <= D2 <= (1 - D3 + D1):
        return "mode_4"
    elif D1 <= D3 <= 1 and (1 - D3) <= D2 <= (1 - D3 + D1):
        return "mode_4_prime"
    elif (D1 - D3) <= D2 <= (1 - D3) and 0 <= D3 <= D1:
        return "mode_5"
    elif (D1 - D3) <= D2 <= (1 - D3) and 0 <= D3 <= D1:
        return "mode_5_prime"
    elif (1 - D2) <= D1 and (1 - D2) <= D3 <= D1:
        return "mode_6"
    elif (1 - D2) <= D1 and (1 - D2) <= D3 <= D1:
        return "mode_6_prime"
    else:
        # 默认模式，如果没有任何匹配条件
        return "mode_1"

# 计算传输功率，返回归一化后的结果
def calculate_power(mode, D1, D2, D3):
    if mode == "mode_1":
        P_o = 2 * (D2**2 - D1 * D2 + 2 * D2 * D3)
    elif mode == "mode_1_prime":
        P_o = -2 * (D2**2 - D1 * D2 + 2 * D2 * D3)
    elif mode == "mode_2":
        P_o = 2 * (D1**2 - D1 * D2 + 2 * D1 - 2 * D1 * D3)
    elif mode == "mode_2_prime":
        P_o = -2 * (D1**2 - D1 * D2 + 2 * D1 - 2 * D1 * D3)
    elif mode == "mode_3":
        P_o = 2 * D2 * D3
    elif mode == "mode_3_prime":
        P_o = -2 * D2 * D3
    elif mode == "mode_4":
        P_o = 2 * (-D2**2 - D3**2 + 2 * D2 * D3 + 2 * D3 + 2 * D2 * D3 + D1 * D2 - 1)
    elif mode == "mode_4_prime":
        P_o = -2 * (-D2**2 - D3**2 + 2 * D2 * D3 + 2 * D3 + 2 * D2 * D3 + D1 * D2 - 1)
    elif mode == "mode_5":
        P_o = 2 * (D1 - D3) * D2 + D3 + 2 * D2 * D3
    elif mode == "mode_5_prime":
        P_o = -2 * (D1 - D3) * D2 + D3 + 2 * D2 * D3
    elif mode == "mode_6":
        P_o = 2 * (-D1**2 - D2**2 - D3**2 + 2 * D3 * D2 + D1 + D2 + 2 * D2 * D3 + 2 * D1 - 1)
    elif mode == "mode_6_prime":
        P_o = -2 * (-D1**2 - D2**2 - D3**2 + 2 * D3 * D2 + D1 + D2 + 2 * D2 * D3 + 2 * D1 - 1)
    else:
        P_o = 0
    return P_o

# 计算峰值电流并归一化
def calculate_peak_current(mode, D1, D2, D3):
    if mode == "mode_1":
        currents = [
            -k * D1 + D2,
            -k * D1 + 2 * k * D3 + D2,
            -k * D1 + 2 * k * D2 + 2 * k * D3 - D2,
            k * D1 - D2
        ]
    elif mode == "mode_1_prime":
        currents = [
            -k * D1 - D2,
            -k * D1 + 2 * k * D3 - D2,
            -k * D1 + 2 * k * D2 + 2 * k * D3 + D2,
            k * D1 + D2
        ]
    elif mode == "mode_2":
        currents = [
            -k * D1 + 2 * D1 - D2 - 2 * D3,
            k * D1 + 2 * D1 - D2 + 2 * D3,
            k * D1 + D2,
            k * D1 + D2
        ]
    elif mode == "mode_2_prime":
        currents = [
            -k * D1 - 2 + D2 + 2 * D3,
            k * D1 - 2 + D2 + 2 * D3,
            k * D1 - D2,
            k * D1 - D2
        ]
    elif mode == "mode_3":
        currents = [
            -k * D1 + D2,
            k * D1 + D2,
            k * D1 + D2,
            k * D1 + D2
        ]
    elif mode == "mode_3_prime":
        currents = [
            -k * D1 - D2,
            k * D1 - D2,
            k * D1 - D2,
            k * D1 + D2
        ]
    elif mode == "mode_4":
        currents = [
            -k * D1 + 2 + D2 - 2 * D3,
            -k * D1 - 2 * k * D2 + 2 * k * D3 + D2,
            k * D1 + D2,
            k * D1 - D2
        ]
    elif mode == "mode_4_prime":
        currents = [
            -k * D1 - 2 + D2 + 2 * D3,
            k * D1 - 2 + k * D2 - k * D3,
            k * D1 - D2,
            k * D1 - D2
        ]
    elif mode == "mode_5":
        currents = [
            -k * D1 + D2,
            k * D1 - 2 * D1 + D2 + 2 * D3,
            k * D1 - D2,
            k * D1 - D2
        ]
    elif mode == "mode_5_prime":
        currents = [
            -k * D1 - D2,
            k * D1 + 2 * D1 - D2 - 2 * D3,
            k * D1 - D2,
            k * D1 + D2
        ]
    elif mode == "mode_6":
        currents = [
            -k * D1 - D2 - 2 * D3 + 2,
            -k * D1 + 2 * k * D3 + D2 - k,
            k * D1 - 2 * D1 + D2 + 2 * D3,
            k * D1 - 2 * D1 + D2 + 2 * D3
        ]
    elif mode == "mode_6_prime":
        currents = [
            -k * D1 + D2 + 2 * D3 - 2,
            -k * D1 + 2 * k * D2 + 2 * k * D3 - D2,
            k * D1 + 2 * D1 - D2 + 2 * D3,
            k * D1 + D2
        ]
    else:
        currents = [0]

    # 取绝对值最大的电流作为峰值电流，并进行归一化
    peak_current = max(abs(i) for i in currents) # * normalization_factor
    return peak_current, currents  # 返回电流列表以用于 ZVS 检查

# 检查当前状态是否满足给定模式的 ZVS 约束
def check_mode_constraints(mode, currents):
    # 根据模式定义 ZVS 约束条件
    if mode == "mode_1":
        return currents[0] <= 0 and currents[1] == 0 and currents[2] == 0
    elif mode == "mode_1_prime":
        return currents[0] <= 0 and currents[1] <= 0 and currents[2] >= 0
    elif mode == "mode_2":
        return currents[0] <= 0 and currents[1] >= 0 and currents[2] >= 0
    elif mode == "mode_2_prime":
        return currents[0] <= 0 and currents[1] >= 0 and currents[2] <= 0
    elif mode == "mode_3":
        return currents[0] == 0 and currents[1] >= 0
    elif mode == "mode_3_prime":
        return currents[0] <= 0 and currents[1] == 0
    elif mode == "mode_4":
        return currents[0] <= 0 and currents[1] >= 0 and currents[2] >= 0
    elif mode == "mode_4_prime":
        return currents[0] <= 0 and currents[1] <= 0 and currents[2] == 0
    elif mode == "mode_5":
        return currents[0] == 0 and currents[1] >= 0 and currents[2] >= 0
    elif mode == "mode_5_prime":
        return currents[0] <= 0 and currents[1] <= 0 and currents[2] >= 0
    elif mode == "mode_6":
        return currents[0] <= 0 and currents[1] >= 0 and currents[2] >= 0 and currents[3] >= 0
    elif mode == "mode_6_prime":
        return currents[0] <= 0 and currents[1] <= 0 and currents[2] <= 0 and currents[3] >= 0
    else:
        return False


# 奖励函数
def get_reward(G_c, G_p):
    delta_G = G_c - G_p
    if G_c <= G_min:
        return 20
    elif delta_G >= 0 and delta_G < G_ref:
        return -abs(delta_G) / G_ref
    elif delta_G < 0:
        return 1
    else:
        return -1

# 计算目标函数 G
def calculate_objective_function(I_p, P_o, zvs_satisfied, beta_zvs=1, beta_non_zvs=10, lambda_=50):
    beta = beta_zvs if zvs_satisfied else beta_non_zvs
    power_error = abs(P_o - P_on)
    return beta * I_p + lambda_ * power_error

# Q-learning 的 ε-greedy 策略
def epsilon_greedy_policy(state_index, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, len(actions) - 1)
    else:
        return np.argmax(Q_table.get(state_index, np.zeros(len(actions))))

# 训练函数
def train(V1, V2, P_o_star):
    global G_min, Q_table, P_on, k
    G_min = G_min_initial
    Q_table = {}
    k = V1 / (n * V2)
    P_max = n * V1 * V2 / (8 * fs * Lk)
    P_on = P_o_star / P_max
    normalization_factor = update_normalization_factor(V2)
    
    min_objective_value = float('inf')
    min_objective_state = None

    for episode in range(N_T):
        D1 = round(random.uniform(-1, 1) / delta) * delta
        D2 = round(random.uniform(-1, 1) / delta) * delta
        D3 = round(random.uniform(-1, 1) / delta) * delta

        mode = determine_mode(D1, D2, D3)
        total_reward = 0
        consecutive_G_min_count = 0

        I_p, currents = calculate_peak_current(mode, D1, D2, D3)
        P_o = calculate_power(mode, D1, D2, D3)
        zvs_satisfied = check_mode_constraints(mode, currents)
        G_p = calculate_objective_function(I_p, P_o, zvs_satisfied)

        for step in range(N_i):
            epsilon = max(0.1, 1.0 - episode / M)
            state_index = get_state_index(D1, D2, D3)
            if state_index not in Q_table:
                Q_table[state_index] = np.zeros(len(actions))
                
            action_index = epsilon_greedy_policy(state_index, epsilon)
            action = actions[action_index]
            
            new_D1, new_D2, new_D3 = D1 + action[0], D2 + action[1], D3 + action[2]
            mode = determine_mode(new_D1, new_D2, new_D3)
            I_p, new_currents = calculate_peak_current(mode, new_D1, new_D2, new_D3)
            P_o = calculate_power(mode, new_D1, new_D2, new_D3)
            zvs_satisfied = check_mode_constraints(mode, new_currents)
            G_c = calculate_objective_function(I_p, P_o, zvs_satisfied)

            if G_c < min_objective_value:
                min_objective_value = G_c
                min_objective_state = (new_D1, new_D2, new_D3)

            reward = get_reward(G_c, G_p)
            total_reward += reward
            next_state_index = get_state_index(new_D1, new_D2, new_D3)
            update_q_value(state_index, action_index, reward, next_state_index)
            G_p = G_c

            if G_c <= G_min:
                G_min = G_c
                consecutive_G_min_count += 1
                if consecutive_G_min_count >= M:
                    return min_objective_state, min_objective_value
            else:
                consecutive_G_min_count = 0
            
            D1, D2, D3 = new_D1, new_D2, new_D3

    return min_objective_state, min_objective_value

# 主运行函数，遍历多组 (V1, V2, P_o)
def run_experiments(V1_values, V2_values, P_o_values, output_file="results.csv"):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["V1", "V2", "P_o", "Optimal D1", "Optimal D2", "Optimal D3", "Min Objective Value"])

        for V1 in V1_values:
            for V2 in V2_values:
                for P_o in P_o_values:
                    min_state, min_value = train(V1, V2, P_o)
                    D1, D2, D3 = min_state if min_state else (None, None, None)
                    D1, D2, D3 = round(D1, 3), round(D2, 3), round(D3, 3)
                    writer.writerow([V1, V2, P_o, D1, D2, D3, min_value])
                    print(f"V1={V1}, V2={V2}, P_o={P_o} -> Optimal (D1, D2, D3): ({D1}, {D2}, {D3}), Min Objective Value: {min_value}")

# 运行实验并输出结果
V1_values = np.arange(100, 140.5, 0.5) #[100,140]
V2_values = np.arange(40, 50.5, 0.5) #[40,50]
P_o_values = np.arange(0, 200.5, 0.5) #[0,200]
run_experiments(V1_values, V2_values, P_o_values)


V1=100.0, V2=40.0, P_o=0.0 -> Optimal (D1, D2, D3): (0.0, 0.0, 0.51), Min Objective Value: 0.0
V1=100.0, V2=40.0, P_o=0.5 -> Optimal (D1, D2, D3): (0.02, 0.05, 0.0), Min Objective Value: 0.06249999999997272
V1=100.0, V2=40.0, P_o=1.0 -> Optimal (D1, D2, D3): (0.0, 0.01, 0.21), Min Objective Value: 0.1049999999999764
V1=100.0, V2=40.0, P_o=1.5 -> Optimal (D1, D2, D3): (-0.0, 0.01, 0.31), Min Objective Value: 0.10249999999996866
V1=100.0, V2=40.0, P_o=2.0 -> Optimal (D1, D2, D3): (-0.0, 0.01, 0.41), Min Objective Value: 0.09999999999999984
V1=100.0, V2=40.0, P_o=2.5 -> Optimal (D1, D2, D3): (0.0, 0.01, 0.51), Min Objective Value: 0.10249999999996813
V1=100.0, V2=40.0, P_o=3.0 -> Optimal (D1, D2, D3): (0.02, 0.05, 0.01), Min Objective Value: 0.09499999999996356
V1=100.0, V2=40.0, P_o=3.5 -> Optimal (D1, D2, D3): (-0.0, 0.01, 0.72), Min Objective Value: 0.10249999999993843
V1=100.0, V2=40.0, P_o=4.0 -> Optimal (D1, D2, D3): (0.0, 0.01, 0.82), Min Objective Value: 0.09999999999999998
V1=100